In [ ]:
import cv2
import json
import numpy as np
# from scipy.spatial import distance
# import tensorflow as tf
# from tensorflow.keras.models import load_model
# from tensorflow.keras.preprocessing import image
import Facenet
# from deepface import DeepFace
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# An input file to get stored embeddings
input_file = 'embeddings/embeddings.json'

model_file = 'models/facenet_weights.h5'

cv_model_file = 'models/haarcascade_frontalface_default.xml'

WIDTH = 640
HEIGHT = 480

# Set the cosine similarity threshold for recognition
threshold = 0.7  # Adjust this value as needed

In [ ]:
stored_embeddings = []

# Load the embeddings from the JSON file
with open(input_file, 'r') as json_file:
    stored_data = json.load(json_file)

    # Create a dictionary to store the embeddings by name
    stored_embeddings = {}
    for item in stored_data:
        name = item['name']
        embeddings = item['embeddings']
        stored_embeddings[name] = []
        for timestamp, embedding in embeddings.items():
            stored_embeddings[name].append(embedding)

print(stored_embeddings)

In [ ]:
# Extract faces
def extract_faces(img_path):
    print(f"Extracting faces for image: {img_path}")
    imgs = DeepFace.extract_faces(img_path=img_path, enforce_detection=False)
    return imgs

# Create embeddings
def create_embeddings(img_path):
    # print(f"Creating embeddings for image: {img_path}")
    print(f"Creating embeddings...")
    embedding_objs = DeepFace.represent(img_path=img_path, model_name="Facenet", enforce_detection=False)
    return embedding_objs

In [ ]:
def main():
    # Load the FaceNet model
    face_net_model = Facenet.loadModel()
    # Load a pre-trained face detection model (e.g., Haar Cascade or dlib)
    face_cascade = cv2.CascadeClassifier(cv_model_file)

    # Create a VideoCapture object to capture video from the default camera (usually 0)
    cap = cv2.VideoCapture(0)
    cap.set(3,WIDTH)
    cap.set(4,HEIGHT)

    while True:
        # Read a frame from the camera
        ret, frame = cap.read()

        # Display the frame in a window named 'Video Capture'
        # cv2.imshow('Video Capture', frame)

        # Detect faces in the frame
        faces = face_cascade.detectMultiScale(frame, scaleFactor=1.1, minNeighbors=5, minSize=(50, 50))

        for (x, y, w, h) in faces:
            # Extract the face ROI
            face_roi = frame[y:y+h, x:x+w]
            
            # Preprocess the face image for FaceNet
            face_img = cv2.resize(face_roi, (160, 160))
            face_img = np.expand_dims(face_img, axis=0)
            face_img = (face_img - 127.5) / 128.0  # Normalize

            # Extract embeddings from the face using FaceNet
            face_embeddings = face_net_model.predict(face_img)
            # face_embeddings = []
            # embeddings = create_embeddings(frame)
            # if len(embeddings) > 0:
            #     for embedding in embeddings:
            #         face_embeddings.append(embedding["embedding"])
            
            # Compare the embeddings with stored embeddings for each user
            min_distance = float("inf")
            recognized_face = None

            for name, stored_user_embeddings in stored_embeddings.items():
                for stored_embedding in stored_user_embeddings:
                    # # Calculate the distance between the embeddings
                    # dist = distance.euclidean(stored_embedding, face_embeddings[0])
                    # Calculate the Euclidean distance between the embeddings
                    dist = np.linalg.norm(stored_embedding - face_embeddings[0])
                    
                    if dist < min_distance:
                        min_distance = dist
                        recognized_face = name
            
            print("distance: ", min_distance)
            if min_distance > threshold:  # You can set a threshold for recognition
                cv2.putText(frame, recognized_face, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
            else:
                cv2.putText(frame, 'Unknown', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)

            # # Compare the embeddings with stored embeddings for each user using cosine similarity
            # max_similarity = -1.0
            # recognized_face = None

            # for name, stored_user_embeddings in stored_embeddings.items():
            #     for stored_embedding in stored_user_embeddings:
            #         # Calculate cosine similarity between the embeddings
            #         similarity = cosine_similarity([stored_embedding], [face_embeddings[0]])[0][0]
                    
            #         if similarity > max_similarity:
            #             max_similarity = similarity
            #             recognized_face = name
            
            # if max_similarity > threshold:  # Check against the threshold for recognition
            #     cv2.putText(frame, recognized_face, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
            # else:
            #     cv2.putText(frame, 'Unknown', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)

        cv2.imshow('Face Recognition', frame)

        # Listen for a key press and check if it's the "Esc" key (27 is the ASCII code for Esc)
        key = cv2.waitKey(1)
        if key == 27:
            break

    # Release the VideoCapture and close all OpenCV windows
    cap.release()
    cv2.destroyAllWindows()

main()
